# Keras 
### An API specification that's used for defining and training machine learning models


### Keras.Sequential  API - CNN example 

1.  tf.Keras.Sequential builds a tf.keras.Model object by stacking Keras layers
2.  model.compile - creates a training loop 
3.  model.fit - executes the training loop 



In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10
from tensorflow.keras import layers


### Prepare data 

In [38]:
(train, train_labels), (test, test_labels) = fashion_mnist.load_data()
# (train, train_labels), (test, test_labels) = cifar10.load_data()

print(f"Training Set: {train.shape}")
print(f"Test Set: {test.shape}")
print(f"Input shape: {train[0].shape}")

input_shape = test[0].shape



Training Set: (60000, 28, 28)
Test Set: (10000, 28, 28)
Input shape: (28, 28)


### Build CNN classification model 

In [41]:
number_of_classes = 10
# build a tf.keras.Model object by stacking Keras layers 
model = tf.keras.Sequential(name="Net")

# first layer, if no input layer is specified the ouputs (shapes) of the subsequent 
# layers is unknown and summary can not be generated. 
model.add(layers.Conv2D(32, (5,5), activation=tf.nn.relu, input_shape=(28,28,1), name="KS_CONV_1")) 
model.add(layers.MaxPool2D((2,2), (2,2), name="KS_POOL_1")) 
model.add(layers.Conv2D(64, (3,3), activation=tf.nn.relu, name="KS_CONV_2")) 
model.add(layers.MaxPool2D((2,2), (2,2), name="KS_POOL_2"))
model.add(layers.Conv2D(32, (3,3), activation=tf.nn.relu, name="KS_CONV_3"))
model.add(layers.MaxPool2D((2,2), (2,2), name="KS_POOL_3"))
model.add(layers.Flatten(name="KS_FLATTEN"))
model.add(layers.Dense(1024, activation=tf.nn.relu, name="KS_FC_1"))
tf.keras.layers.Dropout(0.5, name="KS_DROPOUT")
tf.keras.layers.Dense(number_of_classes, name="KS_FC_OUTPUT")

model.summary()




Model: "Net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
KS_CONV_1 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
KS_POOL_1 (MaxPooling2D)     (None, 12, 12, 32)        0         
_________________________________________________________________
KS_CONV_2 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
KS_POOL_2 (MaxPooling2D)     (None, 5, 5, 64)          0         
_________________________________________________________________
KS_CONV_3 (Conv2D)           (None, 3, 3, 32)          18464     
_________________________________________________________________
KS_POOL_3 (MaxPooling2D)     (None, 1, 1, 32)          0         
_________________________________________________________________
KS_FLATTEN (Flatten)         (None, 32)                0       

### Preprocess data, start training loop 

In [43]:
# Scale input in [-1, 1] range
# preprocess using eager execution.
train = train / 255. * 2 - 1
test = test / 255. * 2 - 1
train = tf.expand_dims(train, -1).numpy()
test = tf.expand_dims(train, -1).numpy()

optimizer = tf.keras.optimizers.Adam(1e-5)
# optimizer = tf.keras.optimizers.SGD(1e-5)
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

# create a training loop 
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# execute the training loop 
model.fit(train, train_labels, epochs=10)
model.evaluate(test, test_labels)


Epoch 1/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3171 - accuracy: 0.1026
Epoch 2/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3031 - accuracy: 0.1044
Epoch 3/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3029 - accuracy: 0.1050
Epoch 4/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3022 - accuracy: 0.1068
Epoch 5/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3021 - accuracy: 0.1094
Epoch 6/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3018 - accuracy: 0.1064
Epoch 7/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3014 - accuracy: 0.1097
Epoch 8/10
1875/1875 [==============================] - 21s 11ms/step - loss: 2.3010 - accuracy: 0.1095
Epoch 9/10
1875/1875 [==============================] - 20s 11ms/step - loss: 2.3003 - accuracy: 0.1121
Epoch 10/10
1875/1875 [==============================] - 20s 11m

ValueError: Data cardinality is ambiguous:
  x sizes: 60000
  y sizes: 10000
Please provide data which shares the same first dimension.

# Keras Functional API 



1. The functional API allows you to build complex models
2. Multi-input, multi-output models, easily sharing layers
3. Residual connections 
4. In general define a model with arbitrary and complex topologies 
5. 

In [23]:
input_shape = (100, )

inputs = tf.keras.Input(input_shape)
net = tf.keras.layers.Dense(units=64, activation=tf.nn.relu, name='FC1')(inputs)
net = tf.keras.layers.Dense(units=64, activation=tf.nn.relu, name='FC2')(net)

net = tf.keras.layers.Dense(units=1, name='G')(net)
model= tf.keras.Model(inputs=inputs, outputs=net)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
FC1 (Dense)                  (None, 64)                6464      
_________________________________________________________________
FC2 (Dense)                  (None, 64)                4160      
_________________________________________________________________
G (Dense)                    (None, 1)                 65        
Total params: 10,689
Trainable params: 10,689
Non-trainable params: 0
_________________________________________________________________


# The subclassing method

1. The Sequential and Functional APIs cover almost any possible scenario. 
2. Subclassing can be more flexible, but error-prone and harder to debug.
3. recommended since it separates the layer definition from its usage, making it easy to make mistakes while refactoring the code.



In [34]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.dense_1 = layers.Dense(
        units=64, activation=tf.nn.elu, name="fc1")
        self.dense_2 = layers.Dense(
        units=64, activation=tf.nn.elu, name="fc2")
        self.output = layers.Dense(units=1, name="G")

    def call(self, inputs):
        # Build the model in functional style here
        # and return the output tensor
        net = self.dense_1(inputs)
        net = self.dense_2(net)
        net = self.output(net)

        return net        